# DRQN Crop Yield Prediction 🧠🌾
This notebook loads crop yield data, preprocesses it, and trains a CNN+LSTM-based DRQN model for predicting yield.

In [20]:
# ⚙️ GPU Setup
import tensorflow as tf
from tensorflow.python.client import device_lib

print("✔️ TensorFlow version:", tf.__version__)
print("💻 Available devices:")
print(device_lib.list_local_devices())

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU is available and memory growth is enabled")
    except RuntimeError as e:
        print("❌ GPU setup failed:", e)
else:
    print("⚠️ No GPU found. Using CPU instead.")

✔️ TensorFlow version: 2.19.0
💻 Available devices:
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 168083532410087117
xla_global_id: -1
]
⚠️ No GPU found. Using CPU instead.


In [21]:
# 📦 Imports
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping

In [22]:
# Load the custom 2-row dataset
df = pd.read_csv("../DataSet/crop_data.csv")
df.head()

,Crop,Crop_Year,Season,State,Area,Production,Annual_Rainfall,Fertilizer,Pesticide,Yield
0,Arecanut,1997,Whole Year,Assam,73814.0,56708,2051.4,7024878.38,22882.34,0.796087
1,Arhar/Tur,1997,Kharif,Assam,6637.0,4685,2051.4,631643.29,2057.47,0.710435
2,Castor seed,1997,Kharif,Assam,796.0,22,2051.4,75755.32,246.76,0.238333
3,Coconut,1997,Whole Year,Assam,19656.0,126905000,2051.4,1870661.52,6093.36,5238.051739
4,Cotton(lint),1997,Kharif,Assam,1739.0,794,2051.4,165500.63,539.09,0.420909


In [23]:
for col in ["Crop", "Season", "State"]:
    df[col] = df[col].astype(str).str.strip()

# 🔠 Encode categorical features
encoders = {
    'Crop': LabelEncoder(),
    'Season': LabelEncoder(),
    'State': LabelEncoder()
}
for col, encoder in encoders.items():
    df[col] = encoder.fit_transform(df[col])

In [24]:
# ⚙️ Feature Scaling
features = ['Crop', 'Season', 'State', 'Area', 'Annual_Rainfall', 'Fertilizer', 'Pesticide']
target = 'Yield'

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df[features])
y = df[target].values

In [25]:
# ⏱️ Sequence creator for RNN
def create_sequences(X, y, window=3):
    Xs, ys = [], []
    for i in range(len(X) - window):
        Xs.append(X[i:i+window])
        ys.append(y[i+window])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_scaled, y)
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [26]:
# 🧠 Build DRQN (CNN + LSTM)
model = Sequential([
    Conv1D(64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    LSTM(64),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.summary()

d:\Environament\envs\agri\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 2, 64)          │           960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,097 (141.00 KB)

 Trainable params: 36,097 (141.00 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
# 🏋️ Train the model
early_stop = EarlyStopping(patience=10, restore_best_weights=True)
model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.2, callbacks=[early_stop])

Epoch 1/100
788/788 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 703217.1250 - val_loss: 911019.7500
Epoch 2/100
788/788 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 631395.3125 - val_loss: 891212.0625
Epoch 3/100
788/788 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 776046.3750 - val_loss: 876932.6875
Epoch 4/100
788/788 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 631950.6250 - val_loss: 864524.3750
Epoch 5/100
788/788 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 722284.5000 - val_loss: 858232.8750
Epoch 6/100
788/788 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 688129.6250 - val_loss: 856924.5000
Epoch 7/100
788/788 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 656719.3125 - val_loss: 843034.6875
Epoch 8/100
788/788 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 682041.3750 - val_loss: 838015.5625
Epoch 9/100
788/788 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 657831.0000 - val_loss: 831930.7500
Epoch 10/100
788/788 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 811133.7500 - val_loss: 825974.6250
Epoch 11/100
788/788 ━━━━━━━━

In [28]:
# 💾 Save model and preprocessors
os.makedirs("../Models/CropYield", exist_ok=True)
model.save("../Models//CropYield/model.h5")
with open("../Models/CropYield/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)
with open("../Models/CropYield/encoders.pkl", "wb") as f:
    pickle.dump(encoders, f)